In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

/home/kuptservol/.venv/yandex/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# bs=48
bs=128

In [3]:
path = f'/home/kuptservol/.kaggle/data/jigsaw-toxic-comment-classification-challenge'

In [4]:
! ls {path}

sample_submission.csv  test.csv  test_labels.csv  train.csv


In [5]:
train = pd.read_csv(f'{path}/train.csv')
test = pd.read_csv(f'{path}/test.csv')
subm = pd.read_csv(f'{path}/sample_submission.csv')

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train['comment_text'][19]

"Don't mean to bother you \n\nI see that you're writing something regarding removing anything posted here and if you do oh well but if not and you can acctually discuss this with me then even better.\n\nI'd like to ask you to take a closer look at the Premature wrestling deaths catagory and the men listed in it, surely these men belong together in some catagory. Is there anything that you think we can do with the catagory besides delting it?"

In [8]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805,0.898321
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420,0.302226
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [10]:
len(train),len(test)

(159571, 153164)

## LSTM pretrain encoder

In [30]:
from fastai.text import *

In [31]:
bs=64

In [32]:
df = pd.concat([train,test], sort=False)

In [33]:
data_lm = (TextList.from_df(df, path, cols=COMMENT)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [34]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=1.)

In [35]:
lr = 1e-2
lr *= bs/48

In [35]:
learn_lm.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [16]:
data_class = (TextList.from_df(train, path, vocab=data_lm.vocab, cols=COMMENT)
        .split_by_rand_pct(0.1, seed=42)
        .label_from_df(cols=label_cols)
        .databunch(bs=bs, num_workers=1))

In [23]:
learn = text_classifier_learner(data_class, AWD_LSTM, drop_mult=0.5)

In [24]:
learn

RNNLearner(data=TextClasDataBunch;

Train: LabelList (143614 items)
x: TextList
xxbos xxmaj explanation 
  xxmaj why the edits made under my username xxmaj hardcore xxmaj metallica xxmaj fan were reverted ? xxmaj they were n't vandalisms , just closure on some gas after i voted at xxmaj new xxmaj york xxmaj dolls xxup fac . xxmaj and please do n't remove the template from the talk page since i 'm retired xxunk,xxbos xxmaj xxunk ! xxmaj he matches this background colour i 'm seemingly stuck with . xxmaj thanks . ( talk ) 21:51 , xxmaj january 11 , 2016 ( xxup utc ),xxbos xxmaj hey man , i 'm really not trying to edit war . xxmaj it 's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page . xxmaj he seems to care more about the formatting than the actual info .,xxbos " 
  xxmaj more 
  i ca n't make any real suggestions on improvement - i wondered if the section statistics should be later on , or a subsection of " " types o

In [21]:
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

In [29]:
learn.predict('good, good, good, good, good, ')[2]

tensor([0.4946, 0.5299, 0.5216, 0.4970, 0.4755, 0.5269])